In [2]:
import psycopg2
import csv
import os

# Database connection parameters
DB_CONFIG = {
    'dbname': 'final_project',  # Replace with your database name
    'user': 'postgres',         # Replace with your database username
    'password': 'Vijay@_2002',   # Replace with your database password
    'host': 'localhost',        # Replace with your host
    'port': 5432                # Replace with your PostgreSQL port (default is 5432)
}

# Directory to save the CSV reports
REPORTS_FOLDER = 'reports'

# Ensure the directory exists
os.makedirs(REPORTS_FOLDER, exist_ok=True)

def save_report_to_csv(data, columns, filename):
    """Save data to a CSV file."""
    file_path = os.path.join(REPORTS_FOLDER, filename)
    
    # Write data to the CSV file
    with open(file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(columns)  # Write the header
        writer.writerows(data)    # Write the data

    print(f"Report saved to {file_path}")

def fetch_report(query, filename, columns):
    """Fetch data from the database and save it as a CSV report."""
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        cursor = conn.cursor()

        # Execute the query
        cursor.execute(query)

        # Fetch all results
        data = cursor.fetchall()

        # Save the data to a CSV file
        save_report_to_csv(data, columns, filename)

    except psycopg2.Error as e:
        print(f"Error executing query: {e}")
    finally:
        # Close the cursor and connection
        if cursor:
            cursor.close()
        if conn:
            conn.close()

# 1. User Enrollment Report
user_enrollment_query = """
    SELECT 
        u.name AS user_name,
        c.title AS course_title,
        ca.progress,
        ca.assignment_date
    FROM 
        "User" u
    JOIN 
        "CourseAssignment" ca ON u.id = ca.user_id
    JOIN 
        "Course" c ON ca.course_id = c.id;
"""
user_enrollment_columns = ['User Name', 'Course Title', 'Progress (%)', 'Assignment Date']
fetch_report(user_enrollment_query, 'user_enrollment_report.csv', user_enrollment_columns)

# 2. User Performance Report
user_performance_query = """
    SELECT 
        u.name AS user_name,
        c.title AS course_title,
        ca.quiz_score,
        ca.engagement_score,
        ca.assignment_grade,
        pr.rating AS overall_rating
    FROM 
        "User" u
    JOIN 
        "CourseAssignment" ca ON u.id = ca.user_id
    JOIN 
        "Course" c ON ca.course_id = c.id
    LEFT JOIN 
        "PerformanceRating" pr ON ca.id = pr.assignment_id;
"""
user_performance_columns = ['User Name', 'Course Title', 'Quiz Score', 'Engagement Score', 'Assignment Grade', 'Overall Rating']
fetch_report(user_performance_query, 'user_performance_report.csv', user_performance_columns)

# 3. Course Completion and Certification Report
course_completion_query = """
    SELECT 
        u.name AS user_name,
        c.title AS course_title,
        ca.progress,
        cert.is_certified
    FROM 
        "User" u
    JOIN 
        "CourseAssignment" ca ON u.id = ca.user_id
    JOIN 
        "Course" c ON ca.course_id = c.id
    LEFT JOIN 
        "Certificates" cert ON ca.id = cert.assignment_id
    WHERE 
        ca.progress = 100;  -- Only show courses with 100% progress
"""
course_completion_columns = ['User Name', 'Course Title', 'Progress (%)', 'Is Certified']
fetch_report(course_completion_query, 'course_completion_report.csv', course_completion_columns)

# 4. Learning Path Summary Report
learning_path_summary_query = """
    SELECT 
        u.name AS user_name,
        lp.title AS learning_path_title,
        ps.average_rating
    FROM 
        "User" u
    JOIN 
        "PerformanceSummary" ps ON u.id = ps.user_id
    JOIN 
        "LearningPath" lp ON ps.learning_path_id = lp.id;
"""
learning_path_summary_columns = ['User Name', 'Learning Path Title', 'Average Rating']
fetch_report(learning_path_summary_query, 'learning_path_summary_report.csv', learning_path_summary_columns)

# 5. Course Ratings Report
course_ratings_query = """
    SELECT 
        c.title AS course_title,
        AVG(pr.rating) AS average_rating
    FROM 
        "Course" c
    JOIN 
        "CourseAssignment" ca ON c.id = ca.course_id
    JOIN 
        "PerformanceRating" pr ON ca.id = pr.assignment_id
    GROUP BY 
        c.title;
"""
course_ratings_columns = ['Course Title', 'Average Rating']
fetch_report(course_ratings_query, 'course_ratings_report.csv', course_ratings_columns)

# 6. Course Enrollment and Certification Report
course_enrollment_certification_query = """
    SELECT c.title AS course_title, 
           COUNT(ca.id) AS total_enrolled, 
           SUM(CASE WHEN cert.is_certified = true THEN 1 ELSE 0 END) AS total_certified
    FROM "Course" c
    LEFT JOIN "CourseAssignment" ca ON c.id = ca.course_id
    LEFT JOIN "Certificates" cert ON ca.id = cert.assignment_id
    GROUP BY c.title
    ORDER BY total_enrolled DESC;
"""
course_enrollment_certification_columns = ['Course Title', 'Total Enrolled', 'Total Certified']
fetch_report(course_enrollment_certification_query, 'course_enrollment_certification_report.csv', course_enrollment_certification_columns)

# 7. Learning Path Enrollment and Certification Report
learning_path_enrollment_certification_query = """
    SELECT lp.title AS learning_path_title, 
           COUNT(ca.id) AS total_enrolled, 
           SUM(CASE WHEN cert.is_certified = true THEN 1 ELSE 0 END) AS total_certified
    FROM "LearningPath" lp
    LEFT JOIN "CourseLearningPath" clp ON lp.id = clp.learning_path_id
    LEFT JOIN "CourseAssignment" ca ON clp.course_id = ca.course_id
    LEFT JOIN "Certificates" cert ON ca.id = cert.assignment_id
    GROUP BY lp.title
    ORDER BY total_enrolled DESC;
"""
learning_path_enrollment_certification_columns = ['Learning Path Title', 'Total Enrolled', 'Total Certified']
fetch_report(learning_path_enrollment_certification_query, 'learning_path_enrollment_certification_report.csv', learning_path_enrollment_certification_columns)

print("All reports have been generated.")


Report saved to reports\user_enrollment_report.csv
Report saved to reports\user_performance_report.csv
Report saved to reports\course_completion_report.csv
Report saved to reports\learning_path_summary_report.csv
Report saved to reports\course_ratings_report.csv
Report saved to reports\course_enrollment_certification_report.csv
Report saved to reports\learning_path_enrollment_certification_report.csv
All reports have been generated.
